# Compiler for Qiskit circuits into simulated UBQC protocols

In this notebook we present a program compiling Qiskit circuits into UBQC protocols simulated using SquidASM. 

In [3]:
# Import files and libraries

from squidasm.run.stack.config import StackNetworkConfig
from squidasm.run.stack.run import run
import netsquid
from UBQC_client import AliceProgram, LoadType
from UBQC_server import BobProgram
from circuits_qasm import qasm_circs
from qiskit.qasm3 import load


import numpy as np
netsquid.set_qstate_formalism(netsquid.QFormalism.KET)

# Load the test circuits
circuits = qasm_circs()


# Run the simulation
def run_simulation(test_circ=1, draw=False, log=False, input_gates = None, output_gates = None, 
                   loadMethod = LoadType.DEFAULT,loadPath = None, num_times = 100, config = "perfect"
                   ,custom_circ = None):
    
    if(config=="perfect"):
        cfg = StackNetworkConfig.from_file("config_perfect.yaml")
        print("Perfect network configuration chosen!")
    elif(config=="noise"):
        cfg = StackNetworkConfig.from_file("config_noise.yaml") 
        print("Noisy network configuration chosen!")
    elif(config=="default"):
        cfg = StackNetworkConfig.from_file("config_default.yaml")
        print("Default network configuration chosen!")
    else:
        print("Network configuration not valid!")

    args = { "circuit": test_circ, "draw": draw, 
            "log": log, "output": output_gates, "input": input_gates, 
            "loadMethod": loadMethod,"loadPath": loadPath,
            "custom_circ": custom_circ}
    
    if loadMethod == LoadType.DEFAULT :
        circ = circuits[int(args["circuit"])-1]
        print("Client chose circuit {}!".format(int(args["circuit"])))
        print(f"Expected result: {circ[1]}")
        if(args["draw"]):
            print(circ[2])
            
    if input_gates:
        print(f"Client chose input gates {input_gates}!")
        
    if output_gates:
        print(f"Client chose output gates {output_gates}!")
    
    if loadMethod == LoadType.CUSTOM:
        circ = args["custom_circ"]
        #circ = args["loadMethod"]
        print("Client chose custom circuit!")
        if(args["draw"]):
            print(circ.draw())
            
    if loadMethod == LoadType.FILE:
        circ = load(loadPath)
        print("Load Qcircuit from a file")
        if(args["draw"]):
            print(circ.draw())
        
    alice_program = AliceProgram(args)
    bob_program = BobProgram()
    
    meas = []
    meas.append(run(config=cfg,
    programs={"Alice": alice_program, "Bob": bob_program},
    num_times=num_times))

    counter = 0
    if(config!="noise"):    
        results = meas[0][0]
        result = results[0][1]
        results_mat=[]

        for i in range(len(results)):
            results_mat.append(results[i][0])
            if(results[i][0] == result):
                counter += 1
        
    if(config=="noise"):
        results = [meas[0][1][i][0] for i in range(len(meas[0][1]))]
        result = meas[0][1][0][1]
        results_mat = results
        for i in range(len(results)):
            if(results[i] == result):
                counter += 1
            
    print(f"Success rate: {counter} in {len(results)}")
    
    return results_mat

SyntaxError: invalid syntax (<ipython-input-3-d8968cbcb150>, line 62)

## Example usage

#### The only function that needs to get called by the user is run_simulation. Its in- and output can be summarized as follows:

### In:
- $\textbf{test_circ}$ (int): number between 1 and 15 choosing one of the test circuits defined in circuits_qasm.py. Standard is 1.
- $\textbf{draw}$: (bool): if True, the simulated circuit get's drawn. Standard is False.
- $\textbf{log}$ (bool): if True, logging will be enabled. Standard is False.
- $\textbf{input_gates}$ (array): array of input gates (str) that are to be applied before the simulation of the circuit. Standard is None.
- $\textbf{output_gates}$ (array): array of output gates (str) that are to be applied after the simulation of the circuit. Standard is None.
- $\textbf{custom_circ}$ (Qiskit circuit): Customized circuit that get's simulated instead of the test circuits, provided by client. Standard is None.
- $\textbf{num_times}$ (int): Number of simulation iterations. Standard is 100.
- $\textbf{config}$ (string): Either "perfect", "default", or "noise"; determines noise configuration of simulation. Standard is perfect.

### Out:
- $\textbf{Prints success rate}$ (right results out of total number of iterations), not available for custom circuits
- $\textbf{Returns result matrix}$ to the simulation of the UBQC protocol for all simulation runs

In [7]:
# default case
run_simulation(draw=True, test_circ = 9,config="default", num_times = 5,loadMethod = LoadType.DEFAULT )

Default network configuration chosen!
Client chose circuit 9!
Expected result: [0, 1, 1]
       ┌───┐┌───┐     ┌───┐
q10_0: ┤ Z ├┤ H ├──■──┤ X ├
       ├───┤├───┤  │  └─┬─┘
q10_1: ┤ Z ├┤ H ├──┼────┼──
       ├───┤└───┘┌─┴─┐  │  
q10_2: ┤ H ├─────┤ X ├──■──
       └───┘     └───┘     
c10: 3/════════════════════
                           
default case
Measurement in Z-Basis: [0, 1, 1]
default case
Measurement in Z-Basis: [0, 1, 1]
default case
Measurement in Z-Basis: [0, 1, 1]
default case
Measurement in Z-Basis: [0, 1, 1]
default case
Measurement in Z-Basis: [1, 1, 1]
Success rate: 4 in 5


[[0, 1, 1], [0, 1, 1], [0, 1, 1], [0, 1, 1], [1, 1, 1]]

### Running custom circuits

In [3]:
# customized test 1
from qiskit import QuantumRegister,QuantumCircuit, ClassicalRegister
from qiskit.compiler.assembler import assemble

# Building custom circuit
q = QuantumRegister(3)
c = ClassicalRegister(3)
qc = QuantumCircuit(q, c)
qc.z(q[0])
qc.z(q[1])
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])

# Run the simulation, expected result: [0,0,1]
# Disclaimer: Expected result for custom circuit not available; evaluation of success rate has to be performed manually
run_simulation(draw=True,loadMethod = LoadType.CUSTOM ,custom_circ = qc,config='perfect',num_times = 5)

Perfect network configuration chosen!
Client chose custom circuit!
       ┌───┐┌───┐
q98_0: ┤ Z ├┤ H ├
       ├───┤├───┤
q98_1: ┤ Z ├┤ H ├
       ├───┤└───┘
q98_2: ┤ H ├─────
       └───┘     
c98: 3/══════════
                 
custome case
Measurement in Z-Basis: [1, 1, 0]
custome case
Measurement in Z-Basis: [1, 1, 0]
custome case
Measurement in Z-Basis: [1, 1, 0]
custome case
Measurement in Z-Basis: [1, 1, 0]
custome case
Measurement in Z-Basis: [1, 1, 0]
Success rate: 0 in 5


[[1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0]]

In [4]:
# customized test 2
from qiskit import QuantumRegister,QuantumCircuit, ClassicalRegister
from qiskit.compiler.assembler import assemble

# Building custom circuit
q = QuantumRegister(3)
c = ClassicalRegister(3)
qc = QuantumCircuit(q, c)
qc.z(q[0])
qc.z(q[1])
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])

# Run the simulation, expected result: [0,0,1]
# Disclaimer: Expected result for custom circuit not available; evaluation of success rate has to be performed manually
run_simulation(draw=True,loadMethod = LoadType.CUSTOM ,custom_circ = qc,config='perfect',input_gates=['rot_z(128)','rot_z(128)','rot_z(128)']
               ,num_times = 5)

Perfect network configuration chosen!
Client chose input gates ['rot_z(128)', 'rot_z(128)', 'rot_z(128)']!
Client chose custom circuit!
        ┌───┐┌───┐
q179_0: ┤ Z ├┤ H ├
        ├───┤├───┤
q179_1: ┤ Z ├┤ H ├
        ├───┤└───┘
q179_2: ┤ H ├─────
        └───┘     
c179: 3/══════════
                  
custome case
Measurement in Z-Basis: [0, 0, 1]
custome case
Measurement in Z-Basis: [0, 0, 1]
custome case
Measurement in Z-Basis: [0, 0, 1]
custome case
Measurement in Z-Basis: [0, 0, 1]
custome case
Measurement in Z-Basis: [0, 0, 1]
Success rate: 0 in 5


[[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]]

In [2]:
from qiskit import QuantumRegister,QuantumCircuit, ClassicalRegister
from qiskit.compiler.assembler import assemble


# Run the simulation, expected result: [0,0,1]
# Disclaimer: Expected result for custom circuit not available; evaluation of success rate has to be performed manually
run_simulation(draw=True,loadMethod = LoadType.FILE,loadPath = "./qcircuit9.qasm" ,config='perfect',num_times = 3)

Perfect network configuration chosen!
Load Qcircuit from a file
                   ┌───┐┌───┐     ┌───┐
esc__all_qubits_0: ┤ Z ├┤ H ├──■──┤ X ├
                   ├───┤├───┤  │  └─┬─┘
esc__all_qubits_1: ┤ Z ├┤ H ├──┼────┼──
                   ├───┤└───┘┌─┴─┐  │  
esc__all_qubits_2: ┤ H ├─────┤ X ├──■──
                   └───┘     └───┘     
             c0: 3/════════════════════
                                       
load case, not yet done
default case
Measurement in Z-Basis: [0]
load case, not yet done
default case
Measurement in Z-Basis: [0]
load case, not yet done
default case
Measurement in Z-Basis: [0]
Success rate: 3 in 3


[[0], [0], [0]]


### 